In [77]:
import pandas as pd
import numpy as np

In [78]:
data=pd.read_csv('data.csv', delimiter=';')
prices=pd.read_csv('prices.csv', delimiter=';')
quantity=pd.read_csv('quantity.csv', delimiter=';')
sample_supplier=pd.read_csv('sample_supplier.csv', delimiter=';')

In [79]:
data.head()
data.rename(columns = {'part_number\tmanufacturer':'connected_data'}, inplace = True)
prices.rename(columns = {'part_number\tprice':'connected_data'}, inplace = True)
data.head()

,connected_data
0,226 920\tLINEX
1,039 042\tSUNSTAR
2,039 056\tSUNSTAR
3,039 063\tSUNSTAR
4,051 893\tKARCHER


In [80]:
prices.head()

,connected_data
0,"000 002\t54,36"
1,"000 004\t5948,21"
2,"000 007\t15999,74"
3,"000 008\t225,01"
4,"000 014\t8,80"


In [81]:
quantity.head()

,000 002\t1
0,000 008\t1
1,000 014\t>10
2,000 014\t8
3,000 014\t7
4,000 014\t>10


In [82]:
sample_supplier.head()

,part_number\tmanufacturer\tprice
0,226 920\tLINEX\t692.93
1,039 042\tSUNSTAR\t531.74
2,039 056\tSUNSTAR\t697.56
3,039 063\tSUNSTAR\t82.75
4,051 893\tKARCHER\t881.6


In [83]:
data[['part number', 'manufacturer']]=data.connected_data.str.split('\t', expand=True)
data.drop('connected_data', axis=1, inplace=True)
data

,part number,manufacturer
0,226 920,LINEX
1,039 042,SUNSTAR
2,039 056,SUNSTAR
3,039 063,SUNSTAR
4,051 893,KARCHER
...,...,...
477561,928 466,KOLBENSCHMIDT
477562,B13 757,ACI
477563,B13 788,ACI
477564,023 109,ZWM CZAJA


In [84]:
prices[['part number', 'price']]=prices.connected_data.str.split('\t', expand=True)
prices.drop('connected_data', axis=1, inplace=True)
prices['price'] = prices['price'].str.replace(',', '.')
prices

,part number,price
0,000 002,54.36
1,000 004,5948.21
2,000 007,15999.74
3,000 008,225.01
4,000 014,8.80
...,...,...
467653,P01 345,254.70
467654,P01 346,386.67
467655,P01 347,682.90
467656,P01 349,61.83


In [85]:
wrong_column_name = list(quantity.columns.values)
quantity.append(wrong_column_name) #we are doing this to avoid losing data
quantity.rename(columns = {wrong_column_name[0]:'connected_data'}, inplace = True)
quantity[['part number', 'quantity']]=quantity.connected_data.str.split('\t', expand=True)
quantity.drop('connected_data', axis=1, inplace=True)
quantity['quantity'] = quantity['quantity'].apply(lambda x: 10 if x == '>10' else int(x))
quantity.head()

C:\Users\Focus\AppData\Local\Temp\ipykernel_27948\776190203.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quantity.append(wrong_column_name) #we are doing this to avoid losing data


,part number,quantity
0,000 008,1
1,000 014,10
2,000 014,8
3,000 014,7
4,000 014,10


In [86]:
data.head()

,part number,manufacturer
0,226 920,LINEX
1,039 042,SUNSTAR
2,039 056,SUNSTAR
3,039 063,SUNSTAR
4,051 893,KARCHER


In [87]:
prices.head()

,part number,price
0,000 002,54.36
1,000 004,5948.21
2,000 007,15999.74
3,000 008,225.01
4,000 014,8.80


In [88]:
merged_data=pd.merge(pd.merge(data,prices,on='part number'), quantity,on='part number')

In [89]:
merged_data

,part number,manufacturer,price,quantity
0,226 920,LINEX,28.20,10
1,226 920,LINEX,28.20,1
2,226 920,LINEX,28.20,2
3,226 920,LINEX,28.20,10
4,039 042,SUNSTAR,50.51,1
...,...,...,...,...
1051555,023 109,ZWM CZAJA,5.18,1
1051556,023 109,ZWM CZAJA,5.18,1
1051557,023 109,ZWM CZAJA,5.18,1
1051558,023 109,ZWM CZAJA,5.18,1


In [90]:
merged_data['price'] = merged_data['price'].astype(float)

In [91]:
print(merged_data.dtypes)

part number      object
manufacturer     object
price           float64
quantity          int64
dtype: object


In [92]:
final_data = merged_data.loc[(merged_data['price'] > 0) | (merged_data['quantity'] > 0)]
final_data.dropna()
final_data.to_csv('final_data.csv', index=False)
final_data.head()

,part number,manufacturer,price,quantity
0,226 920,LINEX,28.20,10
1,226 920,LINEX,28.20,1
2,226 920,LINEX,28.20,2
3,226 920,LINEX,28.20,10
4,039 042,SUNSTAR,50.51,1


In [93]:
brand_list=final_data['manufacturer'].unique()
brand_df = pd.DataFrame({'Brand name': brand_list, 'Product numbers': 0})
brand_df.head()

,Brand name,Product numbers
0,LINEX,0
1,SUNSTAR,0
2,EPS,0
3,RMS,0
4,OSVAT,0


In [94]:
def brands_statistics(final_data, column_name_with_brands):
    #'manufacturer'
    brand_list=final_data[column_name_with_brands].unique()
    brand_df = pd.DataFrame({'Brand name': brand_list, 'Product numbers': 0})
    brand_df
    
    for index, row in brand_df.iterrows():
        brand_name = row['Brand name']
        test_df = final_data.loc[final_data[column_name_with_brands] == brand_name]
        product_numbers = len(test_df)
        brand_df.loc[brand_df['Brand name'] == brand_name, 'Product numbers'] = product_numbers
        
    return(brand_df)

In [ ]:
brands_dataframe=brands_statistics(final_data, 'manufacturer')

In [ ]:
brands_dataframe.to_csv('brands_df.csv', index=False)